# VerbaLex ASR Model

# Prepare Environment


In [1]:
!pip install --upgrade pip
!pip install --upgrade datasets accelerate soundfile librosa evaluate jiwer tensorboard gradio transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

# Linking to Huggingface Hub
Linking this notebook to huggingface hub to enjoy these benefits:
* Integrated version control: you can be sure that no model checkpoint is lost during training.
* Tensorboard logs: track important metrics over the course of training.
* Model cards: document what a model does and its intended use cases.
* Community: an easy way to share and collaborate with the community!

This step is also **required** as we need authentication to use a dataset from Huggingface later to fine tune our base model.

To get the access token for login, head over to your Huggingface profile and press on`Settings`, then `Access Tokens`. Copy your tokens and paste it in the input box when prompted.

In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HF_TOKEN"))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Load Dataset (Base model)
Loading dataset from Mozilla's Common voice dataset to develop a base ASR model.

Dataset link: https://huggingface.co/datasets/mozilla-foundation/common_voice_13_0

The dataset will need to be streamed from huggingface instead of downloading the whole thing. As it is too large for the Kaggle's notebook storage.

In [3]:
from datasets import load_dataset, IterableDatasetDict

dataset = IterableDatasetDict()

dataset["train"] = load_dataset("mozilla-foundation/common_voice_13_0", "en", split="train", use_auth_token=True, streaming=True)
dataset["test"] = load_dataset("mozilla-foundation/common_voice_13_0", "en", split="test", use_auth_token=True, streaming=True)

/opt/conda/lib/python3.10/site-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


/opt/conda/lib/python3.10/site-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Visualize to check that the data is indeed loaded

In [4]:
print(list(dataset["train"].take(1)))
dataset

Reading metadata...: 1013968it [00:34, 29260.06it/s]


[{'client_id': '09cf128e35eee9b49f71a4c58d893f1a5131b5047474704808317562d522272df611f4d104e28a50112fa4002cf4eb2f4520fab52c4b9401313960c12e32a459', 'path': 'en_train_0/common_voice_en_21635524.mp3', 'audio': {'path': 'en_train_0/common_voice_en_21635524.mp3', 'array': array([ 0.00000000e+00, -1.64753278e-14, -5.40568024e-14, ...,
        7.96796940e-08, -1.15487887e-06, -8.86387454e-07]), 'sampling_rate': 48000}, 'sentence': 'This device has a cathode inside an anode wire cage.', 'up_votes': 2, 'down_votes': 0, 'age': '', 'gender': '', 'accent': '', 'locale': 'en', 'segment': '', 'variant': ''}]


IterableDatasetDict({
    train: IterableDataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        n_shards: 26
    })
    test: IterableDataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        n_shards: 1
    })
})

# Data Cleaning
As we can see from the data above, there are a lot of unnecessary data fields. We will drop those columns in the next cell by only selecting the columns that we need.

In [5]:
dataset = dataset.select_columns(['audio', 'sentence'])
dataset

IterableDatasetDict({
    train: IterableDataset({
        features: ['audio', 'sentence'],
        n_shards: 26
    })
    test: IterableDataset({
        features: ['audio', 'sentence'],
        n_shards: 1
    })
})

# Prepare Feature Extractor, Tokenizer and Data
The ASR pipeline can be decomposed into three components.
1. A feature extractor which pre-process the raw audio inputs. Learn more about it [here](https://huggingface.co/blog/fine-tune-whisper#load-whisperfeatureextractor).
2. The model which performs the sequence-to-sequence mapping. 
3. A tokenizer which post-process the model outputs to text format. Learn more about it [here](https://huggingface.co/blog/fine-tune-whisper#load-whispertokenizer).

The Whisper model has already provided a feature extractor, tokenizer as well as a processor. (Which combines the feature extractor and tokenizer, learn more about it [here](https://huggingface.co/blog/fine-tune-whisper#combine-to-create-a-whisperprocessor)) The following section will import those libraries and prepare the pipeline.

In [6]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

# Load feature extractor to process the raw audio inputs.
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

# Load Whisper tokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## This Section is Optional
Optionally, we can verify that the tokenizer correctly encodes the characters by encoding and decoding a sample from the dataset.

### Decode function
There maybe a parameter within the `decode()` function that we are not familiar with, which is `skip_special_tokens`. Special tokens are added during encoding process. The tokenizer appends 'special tokens' at the start and the end of the sentence, including the start/end of the sentence, the language token, as well as any task tokens specified by `WhisperTokenizer.from_pretrained()` function above.

Hence, the `skip_special_tokens` parameter will specify whether do we want to include the 'special tokens' when we decode it into normal text.

In [7]:
input_str = list(dataset["train"].take(1))[0]["sentence"]

# Encoding the sentence
labels = tokenizer(input_str).input_ids

# Decoding the sentence.
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Reading metadata...: 1013968it [00:40, 25137.31it/s]
2024-03-20 14:06:28.725731: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 14:06:28.725839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 14:06:28.851174: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Input:                 This device has a cathode inside an anode wire cage.
Decoded w/ special:    <|startoftranscript|><|en|><|transcribe|><|notimestamps|>This device has a cathode inside an anode wire cage.<|endoftext|>
Decoded w/out special: This device has a cathode inside an anode wire cage.
Are equal:             True


# Prepare data


Since the Whisper model was trained on data with a sampling rate of 16,000, we'll need to resample our data's sampling rate because the data provided by common voice has a sampling rate of 48,000.

In [8]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))


list(dataset["train"].take(1))

Reading metadata...: 1013968it [00:35, 28813.47it/s]


[{'audio': {'path': 'en_train_0/common_voice_en_21635524.mp3',
   'array': array([-1.13686838e-13,  1.47792889e-12,  0.00000000e+00, ...,
           1.59220781e-06,  3.68470864e-07, -8.33906597e-08]),
   'sampling_rate': 16000},
  'sentence': 'This device has a cathode inside an anode wire cage.'}]

After the sampling rate of our data has been resampled to 16,000kHz, we can then proceed to prepare our data for the model.

In [9]:
def prepare_dataset(batch):
    audio = batch["audio"]
    
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

dataset = dataset.map(prepare_dataset)
list(dataset["train"].take(1))

Reading metadata...: 1013968it [00:34, 29453.28it/s]


[{'audio': {'path': 'en_train_0/common_voice_en_21635524.mp3',
   'array': array([-1.13686838e-13,  1.47792889e-12,  0.00000000e+00, ...,
           1.59220781e-06,  3.68470864e-07, -8.33906597e-08]),
   'sampling_rate': 16000},
  'sentence': 'This device has a cathode inside an anode wire cage.',
  'input_features': array([[-0.65115285, -0.65115285, -0.65115285, ..., -0.65115285,
          -0.65115285, -0.65115285],
         [-0.65115285, -0.65115285, -0.65115285, ..., -0.65115285,
          -0.65115285, -0.65115285],
         [-0.65115285, -0.65115285, -0.65115285, ..., -0.65115285,
          -0.65115285, -0.65115285],
         ...,
         [-0.65115285, -0.65115285, -0.65115285, ..., -0.65115285,
          -0.65115285, -0.65115285],
         [-0.65115285, -0.65115285, -0.65115285, ..., -0.65115285,
          -0.65115285, -0.65115285],
         [-0.65115285, -0.65115285, -0.65115285, ..., -0.65115285,
          -0.65115285, -0.65115285]], dtype=float32),
  'labels': [50258,
   50259

In [10]:
dataset = dataset.remove_columns(["audio", "sentence"])
list(dataset["train"].take(1))

Reading metadata...: 1013968it [00:34, 29686.06it/s]


[{'input_features': array([[-0.65115285, -0.65115285, -0.65115285, ..., -0.65115285,
          -0.65115285, -0.65115285],
         [-0.65115285, -0.65115285, -0.65115285, ..., -0.65115285,
          -0.65115285, -0.65115285],
         [-0.65115285, -0.65115285, -0.65115285, ..., -0.65115285,
          -0.65115285, -0.65115285],
         ...,
         [-0.65115285, -0.65115285, -0.65115285, ..., -0.65115285,
          -0.65115285, -0.65115285],
         [-0.65115285, -0.65115285, -0.65115285, ..., -0.65115285,
          -0.65115285, -0.65115285],
         [-0.65115285, -0.65115285, -0.65115285, ..., -0.65115285,
          -0.65115285, -0.65115285]], dtype=float32),
  'labels': [50258,
   50259,
   50359,
   50363,
   5723,
   4302,
   575,
   257,
   17763,
   1429,
   1854,
   364,
   364,
   1429,
   6234,
   17302,
   13,
   50257]}]

# Training and Evaluation

For training, the Huggingface Trainer will handle most of the work for us. All we have to do is:

* Define a data collator. A data collator will take our pre-processed data and prepares PyTorch tensors for the model.
* Evaluation metrics. We will need to define a `compute_metrics` function that computes the word error rate (WER) metric for evaluation.
* Load a pre-trained checkpoint and configure it correctly for training.
* Define training arguments for the Huggingface Trainer to construct a training schedule.

## Defining a Data Collator

In [11]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        
        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch
    
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

## Evaluation Metrics

In [12]:
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer}

## Load a pre-trained checkpoint

In [13]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

model.generation_config.language = "<|en|>"
model.generation_config.task = "transcribe"

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

## Define training arguments

In [14]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-eng-gen",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=1,
    max_steps=10,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=5,
    eval_steps=5,
    logging_steps=5,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [15]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Training


In [16]:
trainer.train() 

Reading metadata...: 1013968it [00:33, 30013.79it/s]
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
5,3.469800,3.774261,21.522430
10,2.726500,2.297799,20.764820


Reading metadata...: 16372it [00:00, 28990.74it/s]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
/opt/conda/lib/python3.10/site-packages/torch/utils/che

TrainOutput(global_step=10, training_loss=3.098151969909668, metrics={'train_runtime': 10951.3139, 'train_samples_per_second': 0.015, 'train_steps_per_second': 0.001, 'total_flos': 4.61736640512e+16, 'train_loss': 3.098151969909668, 'epoch': 1.0})